In [1]:
import json
import requests
import pprint

### Upload Domain Ontology

User's optionally can upload a domain ontology which includes a list of domain concepts that the Text to Triples service can leverage during the extraction process. The domain ontology has to be uploaded to the same "local" semantic graph where text extraction results will be stored. The local graph is defined via the 'localityURI' property.

Request URL: http://localhost:4200/darpa/aske/uploadDomainOntology


Request Payload: { 'baseURI': 'http://aske.ge.com/text2triples#', 'localityURI': 'http://speed-of-sound', 'ontologyAsString': 'ontology_string' }


This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_upload_domain_ontology

In [2]:
upload_domain_ontology_url = 'http://localhost:4200/darpa/aske/uploadDomainOntology'

owl_line = ''
owl_file_path = 'data/sample.owl' 
file = open(owl_file_path, 'r')

for line in enumerate(file):
    line_str =  (line[1].replace('"', '\"'))
    owl_line = owl_line + ' ' + line_str

input_info = { 'baseURI': 'http://aske.ge.com/text2triples#', 'localityURI': 'http://speed-of-sound', 
              'ontologyAsString': owl_line.strip() }

headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(upload_domain_ontology_url, input_info_json, headers=headers)
# pprint.pprint(r.json())

### Text to Triples Main Service

The following code cell demonstrates how to interact with the main Text to Triples (_t2t_) service. _t2t_ parses a blob of text, extracts relevant scientific concepts and equations and adds them to a "local" semantic graph whose [URI](https://en.wikipedia.org/wiki/Uniform_Resource_Identifier) is provided by the user. The service response includes information about the number of scientific concept and equation it was able to extract from user-specified text. 

Request URL: http://localhost:4200/darpa/aske/text2triples

Request Payload: { 'text': 'The force is defined as mass into acceleration', 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_text_to_triples

In [3]:
text_to_triples_service_url = 'http://localhost:4200/darpa/aske/text2triples'

file_path = 'data/speed-of-sound.txt'

text = ''
with open(file_path) as f:
    text = f.read()

locality = 'http://speed-of-sound'

input_info = {'localityURI': locality, 'text': text}
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(text_to_triples_service_url, input_info_json, headers=headers)
# pprint.pprint(r.json())

### Locality Search

One of the main objectives of _t2t_ is to augment scientific models that are extracted from code. This can be accomplished by identifying scientific models that appear in text only or by identifying information about scientific models from text that can help extract better models from code. Locality Search assists in the latter (see the following code block for example usage of locality search). 

The beta release of locality search allows to search using variable names that appear in code. For instance, an ambiguous variable "a" appears in the code and one wishes to get more context about. Assuming the availability of text associated with code (and it's extraction into a local graph using _t2t_), with the help of locality search API, users can look up which scientific models (aka equations) does "a" appear in.

The response from locality search includes the equation in which the variable appears and wherever possible, the scientific concept assciated with the variable as well as scientific concept's augmented semantic type.

Request URL: http://localhost:4200/darpa/aske/equationVariableContext

Request Payload: { 'variableName': 'a', 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_get_equation_var_context


In [4]:
# The following code block assumes contents of "speed of sound" have been processed and extracted 
# into "http://speed-of-sound"

locality_search_service_url = 'http://localhost:4200/darpa/aske/equationVariableContext'

input_info = {'localityURI': 'http://speed-of-sound', 'variableName': 'R'}
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(locality_search_service_url, input_info_json, headers=headers)

# pprint.pprint(r.json())


### Units-based Locality Search

Additional context about the scientific model represented in the code can be obtained by parsing the surrounding comments. These comments often include refrences to the unit associated with the variable in question. Units-based locality search, allows the user to use code comments that potentially contain references to units to identify related concepts and equations from the associated local graph.

While this feature is being made available as part of the beta release, we expect the reponse of this service to evovle. The current response inlcudes the extracted units and it's associated concept, along with a list of equations (equation URIs) in which the concept appears and the variable name associated with the concept.

Request URL: http://localhost:4200/darpa/aske/unitextraction

Request Payload: { 'text': 'variable measurement degree Celsius', 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_get_units_info

In [5]:
# The following code block assumes contents of "speed of sound" have been processed and extracted 
# into "http://speed-of-sound"

units_search_service_url = 'http://localhost:4200/darpa/aske/unitextraction'

input_info = { 'text': 'variable measurement degree Celsius', 'localityURI': 'http://speed-of-sound' }
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(units_search_service_url, input_info_json, headers=headers)

# pprint.pprint(r.json())

### Save Graph

The local semantic graph generated by _t2t_ can be exported using this API. The save graph API returns the local graph as a seralized string in the [N3](https://en.wikipedia.org/wiki/Notation3) format.

Request URL: http://localhost:4200/darpa/aske/saveGraph

Request Payload: { 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_save_graph


In [6]:
# The following code block assumes contents of "speed of sound" have been processed and extracted 
# into "http://speed-of-sound"

save_graph_service_url = 'http://localhost:4200/darpa/aske/saveGraph'

input_info = { 'localityURI': 'http://speed-of-sound' }
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(save_graph_service_url, input_info_json, headers=headers)

# pprint.pprint(r.json())


{'serializationFormat': 'n3',
 'triples': '@prefix ns1: <http://sadl.org/sadlimplicitmodel#> .\n'
            '@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n'
            '@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n'
            '@prefix xml: <http://www.w3.org/XML/1998/namespace> .\n'
            '@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n'
            '\n'
            '<http://aske.ge.com/text2triples#Air> rdfs:label "Air" ;\n'
            '    ns1:matchingClass <http://aske.ge.com/hypersonicsV2#Air> ;\n'
            '    rdfs:subClassOf ns1:ScientificConcept .\n'
            '\n'
            '<http://aske.ge.com/text2triples#Mach> rdfs:label "Mach" ;\n'
            '    ns1:matchingProperty <http://aske.ge.com/hypersonicsV2#mach> '
            ';\n'
            '    rdfs:subClassOf ns1:ScientificConcept .\n'
            '\n'
            '<http://aske.ge.com/text2triples#Machnumber> rdfs:label "Mach '
            'number" ;\n'
            '    

### Clear Graph

The local semantic graph generated by _t2t_ can be cleared/discarded using this API.

Request URL: http://localhost:4200/darpa/aske/clearGraph

Request Payload: { 'localityURI': 'http://speed-of-sound' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_clear_graph

In [12]:
clear_graph_service_url = 'http://localhost:4200/darpa/aske/clearGraph'

input_info = { 'localityURI': 'http://speed-of-sound' }
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(clear_graph_service_url, input_info_json, headers=headers)

pprint.pprint(r.json())

{'message': 'Graph cleared'}


### Text to Ontology

This API allows users to automatically extract an ontology form unstructured text. This API provides a skeleton domain ontology that users can edit and build upon. 

Request URL: http://localhost:4200/darpa/aske/textToOntology

Request Payload: { 'baseURI': 'string', 'serialization': 'string', 'text': 'string' }

This service can be tested via its Swagger Documentation as well: http://localhost:4200/darpa/aske/ui/#!/text2triples/app_text_to_triples_text_to_ontology

In [17]:
text_to_ontology_service_url = 'http://localhost:4200/darpa/aske/textToOntology'

file_path = 'data/wind-turbine.txt'
text = ''
with open(file_path) as f:
    text = f.read()

input_info = { 'baseURI': 'http://aske.ge.com/text2ontology#', 'serialization': 'n3', 'text': text }
headers = {'Content-Type': 'application/json'}

input_info_json = (json.dumps(input_info))

r = requests.post(text_to_ontology_service_url, input_info_json, headers=headers)

pprint.pprint(r.json())

{'serializationFormat': 'n3',
 'triples': '@prefix ns1: <http://sadl.org/sadlimplicitmodel#> .\n'
            '@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n'
            '@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n'
            '@prefix xml: <http://www.w3.org/XML/1998/namespace> .\n'
            '@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n'
            '\n'
            '<http://aske.ge.com/text2ontology#aerodynamic_coefficients> '
            'rdfs:domain <http://aske.ge.com/text2ontology#DragAndLift> ;\n'
            '    rdfs:range '
            '<http://aske.ge.com/text2ontology#AerodynamicCoefficients> .\n'
            '\n'
            '<http://aske.ge.com/text2ontology#alpha_angle> rdfs:domain '
            '<http://aske.ge.com/text2ontology#Attack> ;\n'
            '    rdfs:range <http://aske.ge.com/text2ontology#AlphaAngle> .\n'
            '\n'
            '<http://aske.ge.com/text2ontology#angle> rdfs:domain '
            '<http://aske